In [1]:
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, Dropdown, fixed
import numpy as np

# Load the Excel file into a DataFrame
df = pd.read_excel('simple-land.xlsx')

# Get the last year in the dataset
last_year = df['Year'].max()

# Identify all "Land Use Stock" columns
land_use_columns = [col for col in df.columns if "Land Use Stock" in col]

# Initialize dictionaries to store results for each target variable
coefficients_dict = {}
mape_dict = {}
predictions_dict = {}

# Function to train and store models based on the selected regression method
def train_models(regression_type):
    coefficients_dict.clear()
    mape_dict.clear()
    predictions_dict.clear()
    
    for target_column in land_use_columns:
        coefficients_dict[target_column] = {}
        mape_dict[target_column] = {}
        predictions_dict[target_column] = {}

        for yr in range(2005, 2021):
            # Filter the data for years before yr for training
            df_train = df[df['Year'] < yr]

            # Data from yr onwards for validation and future prediction
            df_validate = df[(df['Year'] >= yr) & (df['Year'] <= last_year)]

            # Select the relevant columns
            features_columns = [col for col in df.columns if "Factors History" in col]
            X_train = df_train[features_columns]
            y_train = df_train[target_column]

            # Choose the regression model based on the dropdown selection
            if regression_type == 'Linear Regression':
                model = LinearRegression()
            elif regression_type == 'Polynomial Regression':
                # Polynomial regression with degree 2
                model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())

            # Train the model
            model.fit(X_train, y_train)

            # Make predictions on the entire dataset (before yr for training and yr onward for validation)
            df[f'Predicted {target_column}'] = model.predict(df[features_columns])

            # Make predictions on the validation set (years yr and onward)
            X_validate = df_validate[features_columns]
            y_validate = df_validate[target_column]
            y_pred = model.predict(X_validate)

            # Store predictions for this year
            predictions_dict[target_column][yr] = df[f'Predicted {target_column}'].copy()

            # Calculate MAPE for Historical Data (before yr)
            historical_mape = mean_absolute_percentage_error(df[df['Year'] < yr][target_column], 
                                                             df[df['Year'] < yr][f'Predicted {target_column}']) * 100

            # Calculate MAPE for Future Data (yr onwards and within the actual data range)
            future_mape = mean_absolute_percentage_error(y_validate, y_pred) * 100

            # Calculate MAPE for the Entire Dataset (up to the last available year)
            entire_mape = mean_absolute_percentage_error(df[df['Year'] <= last_year][target_column], 
                                                         df[df['Year'] <= last_year][f'Predicted {target_column}']) * 100

            # Store MAPE and coefficients
            mape_dict[target_column][yr] = (historical_mape, future_mape, entire_mape)
            coefficients_dict[target_column][yr] = model.named_steps['linearregression'].coef_ if regression_type == 'Polynomial Regression' else model.coef_

# Function to update plots based on slider
def update_plots(yr, regression_type):
    train_models(regression_type)  # Train models with the selected regression method
    
    # 3x3 grid for all plots
    fig, axes = plt.subplots(3, 3, figsize=(15, 10))  # 3x3 grid of subplots
    
    # Plot Land Use Stock charts
    for i, target_column in enumerate(land_use_columns):
        row = i // 3
        col = i % 3
        ax = axes[row, col]
        
        # Plot Historical Data
        ax.plot(df['Year'], df[target_column], label='Historical Data', marker='o', linewidth=1)
        
        # Plot Model Predictions for the selected year
        ax.plot(df['Year'], predictions_dict[target_column][yr], label=f'Model Prediction (trained until {yr})', linestyle='--', linewidth=1)
        
        # Add vertical line to indicate the year until which the model was trained
        ax.axvline(x=yr, color='gray', linestyle='--', label=f'Year {yr}')
        
        # Add labels and title
        ax.set_xlabel('Year', fontsize=8)
        ax.set_ylabel(target_column, fontsize=8)
        ax.set_title(f'{target_column} (trained until {yr})', fontsize=10)
        ax.grid(True)
        ax.legend(fontsize=6)
    
    # Bar chart for Entire MAPE
    ax = axes[2, 1]  # Use the last available cell in the grid
    entire_mapes = []
    labels = []
    
    for target_column in land_use_columns:
        hist_mape, fut_mape, ent_mape = mape_dict[target_column][yr]
        entire_mapes.append(ent_mape)
        labels.append(target_column)
    
    bars = ax.bar(labels, entire_mapes, color='skyblue')
    ax.bar_label(bars, fmt='%.2f%%', padding=3, fontsize=8)
    ax.set_title(f'Entire MAPE for Each Land Use Stock (Year {yr})', fontsize=10)
    ax.set_ylabel('MAPE (%)', fontsize=8)
    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
    
    # Line chart showing MAPE trends over time
    ax = axes[2, 2]  # Use the last available cell in the grid
    for target_column in land_use_columns:
        entire_mapes_over_time = [mape_dict[target_column][year][2] for year in range(2005, 2021)]
        ax.plot(range(2005, 2021), entire_mapes_over_time, label=target_column, marker='o')
    
    # Calculate and plot the aggregated MAPE across all land use stocks
    aggregated_mape = []
    for year in range(2005, 2021):
        aggregated_mape.append(np.mean([mape_dict[target][year][2] for target in land_use_columns]))
    
    ax.plot(range(2005, 2021), aggregated_mape, label='Aggregated MAPE', color='black', linestyle='--', marker='x', linewidth=2)
    
    ax.set_title('MAPE Trends Over Time', fontsize=10)
    ax.set_xlabel('Year', fontsize=8)
    ax.set_ylabel('MAPE (%)', fontsize=8)
    ax.legend(fontsize=6)
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()

# Create a dropdown for selecting the regression method
regression_dropdown = Dropdown(
    options=['Linear Regression', 'Polynomial Regression'],
    value='Linear Regression',
    description='Method:',
    style={'description_width': 'initial'},
    layout={'width': '50%'}
)

# Create a single slider to control all the plots
slider = IntSlider(min=2005, max=2020, step=1, value=2005, description='Year:', style={'description_width': 'initial'}, layout={'width': '80%'})

# Display the interactive plots and slider
interact(update_plots, yr=slider, regression_type=regression_dropdown)


interactive(children=(IntSlider(value=2005, description='Year:', layout=Layout(width='80%'), max=2020, min=200…

<function __main__.update_plots(yr, regression_type)>